# MIDS W261 Machine Learning At Scale

Christopher Llop | christopher.llop@ischool.berkeley.edu <br>
Week 2 | Submission Date:

<span style="color:red">[Placeholder for introduction to assignment]</span>

<span style="color:black"><b>HW2.0.</b> What is a race condition in the context of parallel computation? Give an example.
What is MapReduce?
How does it differ from Hadoop?
Which programming paradigm is Hadoop based on? Explain and give a simple example in code and show the code running.</span>

<span style="color:green"><b>Answer:</b></span>

<span style="color:black"><b>HW2.1. </b> Sort in Hadoop MapReduce


Given as input: Records of the form (integer, “NA”), where integer is any integer, and “NA” is just the empty string.

Output: sorted key value pairs of the form (integer, “NA”); what happens if you have multiple reducers? Do you need additional steps? Explain.

Write code to generate N  random records of the form (integer, “NA”). Let N = 10,000.
Write the python Hadoop streaming map-reduce job to perform this sort.</span>


<span style="color:green"><b>Answer:</b></span>

<span style="color:black"><b>HW2.2.</b></style> Using the Enron data from HW1 and Hadoop MapReduce streaming, write mapper/reducer pair that  will determine the number of occurrences of a single, user-specified word. Examine the word “assistance” and report your results.


   To do so, make sure that
   
   - mapper.py counts all occurrences of a single word, and
   - reducer.py collates the counts of the single word.

CROSSCHECK: >grep assistance enronemail_1h.txt|cut -d$'\t' -f4| grep assistance|wc -l
       8

<span style="color:black"><b>HW2.3.</b></style> Using the Enron data from HW1 and Hadoop MapReduce, write  a mapper/reducer pair that
   will classify the email messages by a single, user-specified word. Examine the word “assistance” and report your results. To do so, make sure that
   
   - mapper.py
   - reducer.py 

   performs a single word multinomial Naive Bayes classification.

<span style="color:black"><b>HW2.4.</b></style> Using the Enron data from HW1 and in the Hadoop MapReduce framework, write  a mapper/reducer pair that
   will classify the email messages using multinomial Naive Bayes Classifier using a list of one or more user-specified words. Examine the words “assistance”, “valium”, and “enlargementWithATypo” and report your results
   To do so, make sure that

   - mapper.py 
   - reducer.py 

   performs the multiple-word multinomial Naive Bayes classification via the chosen list.

<span style="color:black"><b>HW2.5.</b></style>

Using the Enron data from HW1 an in the  Hadoop MapReduce framework, write  a mapper/reducer for a multinomial Naive Bayes Classifier that
   will classify the email messages using  words present. Also drop words with a frequency of less than three (3). How does it affect the misclassifcation error of learnt naive multinomial Bayesian Classifiers on the training dataset.

In [3]:
# HW 1.1: Read through pNaiveBayes and all comments to 
#         become comfortable with the code.

def HW1_1():
    print "done"

HW1_1()

done


<span style="color:silver"><b>HW1.2.</b> Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh will determine the number of occurrences of a single, user-specified word. Examine the word “assistance” and report your results.</span>

   <span style="color:silver">To do so, make sure that:</span>
   - <span style="color:silver">mapper.py counts all occurrences of a single word, and</span>
   - <span style="color:silver">reducer.py collates the counts of the single word.</span>
   
   

In [4]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

# Get input parameters
findword = sys.argv[2]
findfile = sys.argv[1]
term_hits = {}

with open (findfile, "r") as myfile:
    for full_email in myfile:
        try:
            # Spam classification
            is_spam = re.findall("\t([0-1])\t",full_email)[0]

            # Parse out email body for processing. Find body using "tab spam/ham tab"
            # use regex to strip out non alpha-numeric. "don't" will become "dont" which is fine.
            keyword = re.findall("\t[0-1]\t",full_email)[0]
            email_id, is_spam_tabbed, email_body = full_email.partition(keyword)
            email_body = re.sub('[^A-Za-z0-9\s]+', '', email_body)
            email_len = len(email_body.split())

            # Number of hits for search word. Using a dictionary now will allow us to use
            #    a similar Mapper output format when using multiple search terms later 
            #    in this problem set.
            term_hits[findword] = len(re.findall(findword,email_body))
            
            # Print as tuple with unique splitter "|||"
            print "{} ||| {} ||| {}".format(is_spam, email_len, term_hits)

        except:
            pass
            

Overwriting mapper.py


In [5]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import ast

# Get input parameters - list of file names
filelist = sys.argv[1:]

term_sum = {}

# Open each map result
for thisfile in filelist:
    with open (thisfile, "r") as openfile:
        # Process each emaul one at a time
        for processed_email in openfile:
            # Read data for email
            processed_email = processed_email.split(" ||| ")
            count_dict = ast.literal_eval(processed_email[2])
            
            # Fold (sum) together the results from mapping
            for key, value in count_dict.iteritems():
                term_sum[key] = term_sum.get(key,0) + value

# Print results
for key, value in term_sum.iteritems():
    print "The word count for '{}' is {}".format(key, value)


Writing reducer.py


In [6]:
# Use chmod for permissions - Only need to do this once per assignment since we're just overriding the file
!chmod a+x mapper.py
!chmod a+x reducer.py
!chmod a+x pNaiveBayes.sh

In [7]:
# HW 1.2: Create map/reduce pair that determins occurances of a single word
def HW1_2(word="assistance"):
    import re
    
    # Run pNaiveBayes.sh
    !./pNaiveBayes.sh 10 {word}

    # Print the output file contents to screen
    with open ("enronemail_1h.txt.output", "r") as openfile:
        print "Result:", openfile.read()

    # Crosscheck results (data is small enough to use RE in python)
    with open ("enronemail_1h.txt", "r") as myfile:
        print "Check Result:", len(re.findall(word,myfile.read()))
        
HW1_2(word = "assistance")

Result: The word count for 'assistance' is 10

Check Result: 10


<span style="color:silver"><b>HW1.3. </b>Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh
   will classify the email messages by a single, user-specified word. Examine the word “assistance” and report your results. To do so, make sure that</span>
   
   - <span style="color:silver">mapper.py is same as in part (2), and [Note - later instructions removed this criteria]</span>
   - <span style="color:silver">reducer.py performs a single word Naive Bayes classification.</span>

In [8]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

# Get input parameters
findword = sys.argv[2]
findfile = sys.argv[1]

with open (findfile, "r") as myfile:
    for full_email in myfile:
        try:
            # Empty dictionary
            term_hits = {}
            
            # Spam classification
            is_spam = re.findall("\t([0-1])\t",full_email)[0]

            # Parse out email body for processing. Find body using "tab spam/ham tab"
            # use regex to strip out non alpha-numeric. "don't" will become "dont" which is fine for classifying.
            keyword = re.findall("\t[0-1]\t",full_email)[0]
            email_id, is_spam_tabbed, email_body = full_email.partition(keyword)
            email_body = re.sub('[^A-Za-z0-9\s]+', '', email_body)
            # Must process search query and email bodies the same
            findword = re.sub('[^A-Za-z0-9\s]+', '', findword)
            email_len = len(email_body.split())

            # Build counts of term words. Note, we need to get a count of distinct words
            # regardless, so we might as well count all words here in preparation for the next problem 1.3.
            # In an actual industry application, we would not store counts for all words since that would
            # take up more space than required to solve the problem. Here, the focus is on learning MapReduce
            # and Naive Bayes as opposed to this concern.
            for word in list(set(email_body.split())):
                term_hits[word] = len(re.findall(word,email_body))
            
            # Print as tuple with unique splitter "|||"
            print "{} ||| {} ||| {} ||| {} ||| {}".format(re.findall(r'\w+', findword), email_id, is_spam, email_len, term_hits)

        except:
            pass
            

Overwriting mapper.py


In [9]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import ast
import math

filelist = sys.argv[1:]

spam_term_counts = {}
ham_term_counts = {}
word_prob_given_spam = {}
word_prob_given_ham = {}
spam_count = 0
ham_count = 0
spam_len = 0
ham_len = 0
distinct_term_list = []

# Open each file and build Multinomial Naive Bayes model
for thisfile in filelist:
    with open (thisfile, "r") as openfile:
        for processed_email in openfile:
            # Read in tuples created by mapper
            processed_email = processed_email.split(" ||| ")
            search_terms = ast.literal_eval(processed_email[0])
            email_id = processed_email[1]
            is_spam = int(processed_email[2])
            email_len = int(processed_email[3])
            count_dict = ast.literal_eval(processed_email[4])
            
            # Build counts for spam and ham definitions.
            if is_spam:
                for key, value in count_dict.iteritems():
                    spam_term_counts[key] = spam_term_counts.get(key, 0) + value
                spam_count += 1
                spam_len += email_len
            else:
                for key, value in count_dict.iteritems():
                    ham_term_counts[key] = ham_term_counts.get(key, 0) + value
                ham_count += 1
                ham_len += email_len
                
            distinct_term_list = list(set(distinct_term_list + count_dict.keys()))

# Calculate our priors based on the overall ratio of spam to ham
spam_prior = float(spam_count) / (spam_count + ham_count)
ham_prior = 1 - spam_prior
spam_prior = math.log10(spam_prior)
ham_prior = math.log10(ham_prior)

# Calculate our conditional probabilites for the search term using MNB formula
#     term_given_spam = (spam_term_count + 1 for smoothing) / (total count of spam words + total distinct vocab size)
distinct_term_count = len(distinct_term_list)

for term in search_terms:
    word_prob_given_spam[term] = math.log10((spam_term_counts.get(term,0) + 1.0) / (float(spam_len) + distinct_term_count))
    word_prob_given_ham[term] = math.log10((ham_term_counts.get(term,0) + 1.0) / (float(ham_len) + distinct_term_count))

# Open each file and predict. Note - prediction is embaressingly parallel and could
#   be done effectively via Mapping. However, the assignment asks us to solve the problem using
#   the provided pNaiveBayes.sh.
accuracy = []
for thisfile in filelist:
    with open (thisfile, "r") as openfile:
        for processed_email in openfile:
            # Defaults
            pred_spam = 0
            spam_prediction = spam_prior
            ham_prediction = ham_prior

            # Read in tuples created by mapper
            processed_email = processed_email.split(" ||| ")
            email_id = processed_email[1]
            is_spam = int(processed_email[2])
            count_dict = ast.literal_eval(processed_email[4])
            
            # Read in counts to use in prediction
            for term in word_prob_given_spam.keys():
                # Calculate the probability for each class
                spam_prediction += (word_prob_given_spam[term] * count_dict.get(term, 0))
                ham_prediction += (word_prob_given_ham[term] * count_dict.get(term, 0))
                
            # Pick the higher probability
            if spam_prediction > ham_prediction: 
                pred_spam = 1
            
            # Store accuracy in a list
            accuracy.append(1*(pred_spam==is_spam))

            # Print predictions to results file
            print '{}\t{}\t{}'.format(email_id, is_spam, pred_spam)
# Print accuracy
sys.stderr.write("Spam Probs: {}\n".format(word_prob_given_spam))
sys.stderr.write("Ham Probs: {}\n".format(word_prob_given_ham))
sys.stderr.write("Accuracy = {:.2f}\n".format(float(sum(accuracy))/len(accuracy)))


Overwriting reducer.py


In [10]:
# HW 1.3: Create multinomial bayes map/reduce pair that predicts spam/ham using a single word
def HW1_3(terms="assistance"):
    # Run pNaiveBayes.sh
    !./pNaiveBayes.sh 10 "{terms}"
        
HW1_3(terms = "assistance")

Spam Probs: {'assistance': -3.4264028097516}
Ham Probs: {'assistance': -3.799891684656865}
Accuracy = 0.60


<span style="color:silver"><b>HW1.4.</b> Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh
   will classify the email messages by a list of one or more user-specified words. Examine the words “assistance”, “valium”, and “enlargementWithATypo” and report your results
   To do so, make sure that</span>
 

   - <span style="color:silver">mapper.py counts all occurrences of a list of words, and</span>
   - <span style="color:silver">reducer.py performs the multiple-word Naive Bayes classification via the chosen list.</span>

In [11]:
# HW 1.4: Our function for HW 1.3 can also classify multiple words, as requested by problem 1.4
def HW1_4(words = "assistance valium enlargementWithATypo"):
    # Run pNaiveBayes.sh
    !./pNaiveBayes.sh 10 "{words}"
            
HW1_4(words = "assistance valium enlargementWithATypo")

Spam Probs: {'assistance': -3.4264028097516, 'enlargementWithATypo': -4.380645319190925, 'valium': -3.778585327862962}
Ham Probs: {'assistance': -3.799891684656865, 'enlargementWithATypo': -4.277012939376528, 'valium': -4.277012939376528}
Accuracy = 0.63


<span style="color:silver"><b>HW1.5.</b> Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh will classify the email messages by all words present. To do so, make sure that</span>

   - <span style="color:silver">mapper.py counts all occurrences of all words, and</span>
   - <span style="color:silver">reducer.py performs a word-distribution-wide Naive Bayes classification.</span>

In [12]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

# Get input parameters
findword = sys.argv[2]
findfile = sys.argv[1]

star_switch = 0
if findword == "*":
    star_switch = 1

with open (findfile, "r") as myfile:
    for full_email in myfile:
        try:
            # Empty dictionary
            term_hits = {}

            # Spam classification
            is_spam = re.findall("\t([0-1])\t",full_email)[0]

            # Parse out email body for processing. Find body using "tab spam/ham tab"
            # use regex to strip out non alpha-numeric. "don't" will become "dont" which is fine for classifying.
            keyword = re.findall("\t[0-1]\t",full_email)[0]
            email_id, is_spam_tabbed, email_body = full_email.partition(keyword)
            email_body = re.sub('[^A-Za-z0-9\s]+', '', email_body)
            # Must process search query and email bodies the same
            # This is a good place to add logic to use all terms in body as search terms when "*" appears
            if star_switch == 1:
                findword = " ".join(list(set(email_body.split())))
            else:
                findword = re.sub('[^A-Za-z0-9\s]+', '', findword)
            email_len = len(email_body.split())

            # Build counts of term words.
            for word in list(set(email_body.split())):
                term_hits[word] = len(re.findall(word,email_body))

            # Print as tuple with unique splitter "|||"
            print "{} ||| {} ||| {} ||| {} ||| {}".format(re.findall(r'\w+', findword), email_id, is_spam, email_len, term_hits)

        except:
            pass
            

Overwriting mapper.py


In [13]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import ast
import math

filelist = sys.argv[1:]

spam_term_counts = {}
ham_term_counts = {}
word_prob_given_spam = {}
word_prob_given_ham = {}
spam_count = 0
ham_count = 0
spam_len = 0
ham_len = 0
distinct_term_list = []

# Open each file and build Multinomial Naive Bayes model
for thisfile in filelist:
    with open (thisfile, "r") as openfile:
        for processed_email in openfile:
            # Read in tuples created by mapper
            processed_email = processed_email.split(" ||| ")
            search_terms = ast.literal_eval(processed_email[0])
            email_id = processed_email[1]
            is_spam = int(processed_email[2])
            email_len = int(processed_email[3])
            count_dict = ast.literal_eval(processed_email[4])
            
            # Build counts for spam and ham definitions.
            if is_spam:
                for key, value in count_dict.iteritems():
                    spam_term_counts[key] = spam_term_counts.get(key, 0) + value
                spam_count += 1
                spam_len += email_len
            else:
                for key, value in count_dict.iteritems():
                    ham_term_counts[key] = ham_term_counts.get(key, 0) + value
                ham_count += 1
                ham_len += email_len
                
            distinct_term_list = list(set(distinct_term_list + count_dict.keys()))

# Calculate our priors based on the overall ratio of spam to ham
spam_prior = float(spam_count) / (spam_count + ham_count)
ham_prior = 1 - spam_prior
spam_prior = math.log10(spam_prior)
ham_prior = math.log10(ham_prior)

# Calculate our conditional probabilites for the search term using MNB formula
#     term_given_spam = (spam_term_count + 1 for smoothing) / (total count of spam words + total distinct vocab size)
distinct_term_count = len(distinct_term_list)

for term in distinct_term_list:
    word_prob_given_spam[term] = math.log10((spam_term_counts.get(term,0) + 1.0) / (float(spam_len) + distinct_term_count))
    word_prob_given_ham[term] = math.log10((ham_term_counts.get(term,0) + 1.0) / (float(ham_len) + distinct_term_count))

# Open each file and predict. Note - prediction is embaressingly parallel and could
#   be done effectively via Mapping. However, the assignment asks us to solve the problem using
#   the provided pNaiveBayes.sh.
accuracy = []
for thisfile in filelist:
    with open (thisfile, "r") as openfile:
        for processed_email in openfile:
            # Defaults
            pred_spam = 0
            spam_prediction = spam_prior
            ham_prediction = ham_prior

            # Read in tuples created by mapper
            processed_email = processed_email.split(" ||| ")
            email_id = processed_email[1]
            is_spam = int(processed_email[2])
            count_dict = ast.literal_eval(processed_email[4])
            
            # Read in counts to use in prediction
            for term in distinct_term_list:
                # Calculate the probability for each class
                spam_prediction += (word_prob_given_spam[term] * count_dict.get(term, 0))
                ham_prediction += (word_prob_given_ham[term] * count_dict.get(term, 0))
                
            # Pick the higher probability
            if spam_prediction > ham_prediction: 
                pred_spam = 1
            
            # Store accuracy in a list
            accuracy.append(1*(pred_spam==is_spam))

            # Print predictions to results file
            print '{}\t{}\t{}'.format(email_id, is_spam, pred_spam)
# Print accuracy
sys.stderr.write("Accuracy = {:.2f}\n".format(float(sum(accuracy))/len(accuracy)))


Overwriting reducer.py


In [14]:
# HW 1.5: Run on all words. 
def HW1_5():
    # Run pNaiveBayes.sh
    !./pNaiveBayes.sh 10 "*"
            
HW1_5()

Accuracy = 0.96


In [237]:
test = ["1","3","five"]
print str(test)
print 


['1', '3', 'five']
1 3 five


In [125]:
content = "12313313	1	asdad13	1	a13	0	a"
print re.findall("\t([0-1])\t",content)[0]

1


In [174]:
content = "12313313	1	asdad13	1	a13	0	a"
keyword = re.findall("\t[0-1]\t",content)[0]
email_id, is_spam, email_body = content.partition(keyword)
print email_id
print is_spam
print email_body

12313313
	1	
asdad13	1	a13	0	a
